In [24]:
# add imports for python and spark and configure environment

from graphframes import *
from pyspark import *
from pyspark.sql import *

import pandas as pd
import os
import sys
import pyarrow as pa
import pyarrow.parquet as pq
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_SUBMIT_ARGS'] = """--name job_name --master local --conf spark.dynamicAllocation.enabled=true pyspark-shell"""
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages graphframes:graphframes:0.8.1-spark3.0-s_2.12 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages graphframes:graphframes:0.8.1-spark3.0-s_2.12,neo4j-contrib:neo4j-connector-apache-spark_2.12:4.0.1_for_spark_3 pyspark-shell'

spark = SparkSession.builder.appName("DataFrame").getOrCreate()
conf = SparkConf().setMaster("local").setAppName("Assignment 10")
sc = SparkContext.getOrCreate(conf=conf)
sqlContext = SQLContext(sc)

In [25]:
# read bike data and create dataframes for stations and trip data
bikeStations = spark.read.option("header","true")\
.csv("201508_station_data.csv")
tripData = spark.read.option("header","true")\
.csv("201508_trip_data.csv")

In [26]:
#print schema from bikeStations
bikeStations.printSchema()

root
 |-- station_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)
 |-- dockcount: string (nullable = true)
 |-- landmark: string (nullable = true)
 |-- installation: string (nullable = true)



In [27]:
#print schema from tripData
tripData.printSchema()

root
 |-- Trip ID: string (nullable = true)
 |-- Duration: string (nullable = true)
 |-- Start Date: string (nullable = true)
 |-- Start Station: string (nullable = true)
 |-- Start Terminal: string (nullable = true)
 |-- End Date: string (nullable = true)
 |-- End Station: string (nullable = true)
 |-- End Terminal: string (nullable = true)
 |-- Bike #: string (nullable = true)
 |-- Subscriber Type: string (nullable = true)
 |-- Zip Code: string (nullable = true)



In [28]:
# rename columns to create graphframes
stationVertices = bikeStations.withColumnRenamed("name", "id").distinct()
tripEdges = tripData\
.withColumnRenamed("Start Station", "src")\
.withColumnRenamed("End Station", "dst")

In [29]:
# create station graphframe
from graphframes import GraphFrame
stationGraph = GraphFrame(stationVertices, tripEdges)
stationGraph.cache()

22/04/16 15:31:43 WARN CacheManager: Asked to cache already cached data.
22/04/16 15:31:43 WARN CacheManager: Asked to cache already cached data.


GraphFrame(v:[id: string, station_id: string ... 5 more fields], e:[src: string, dst: string ... 9 more fields])

In [30]:
# import expr func and add configuration for timestamps
from pyspark.sql.functions import expr
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

DataFrame[key: string, value: string]

In [31]:
# create ring of 2 length
motifs1 = stationGraph.find("(a)-[ab]->(b); (b)-[ba]->(a)")

In [9]:
# ring of 2 length query
motifs1.selectExpr("*",
"to_timestamp(ab.`Start Date`, 'MM/dd/yyyy HH:mm') as abStart",
"to_timestamp(ba.`Start Date`, 'MM/dd/yyyy HH:mm') as baStart")\
.where("ab.`Bike #` = ba.`Bike #`")\
.where("a.id != b.id")\
.where("abStart < baStart")\
.orderBy(expr("cast(baStart as long) - cast(abStart as long)"))\
.selectExpr("a.id", "b.id", "ab.`Start Date`", "ba.`End Date`")\
.limit(2).show(2, False)

+-----------------+-----------------+---------------+---------------+
|id               |id               |Start Date     |End Date       |
+-----------------+-----------------+---------------+---------------+
|Townsend at 7th  |2nd at South Park|7/15/2015 16:57|7/15/2015 17:12|
|Market at Sansome|2nd at South Park|6/22/2015 17:12|6/22/2015 17:26|
+-----------------+-----------------+---------------+---------------+



In [10]:
# create ring of 3 length
motifs2 = stationGraph.find("(a)-[ab]->(b); (b)-[bc]->(c); (c)-[ca]->(a)")

In [11]:
# ring of 3 length query
motifs2.selectExpr("*",
"to_timestamp(ab.`Start Date`, 'MM/dd/yyyy HH:mm') as abStart",
"to_timestamp(bc.`Start Date`, 'MM/dd/yyyy HH:mm') as bcStart",
"to_timestamp(ca.`Start Date`, 'MM/dd/yyyy HH:mm') as caStart")\
.where("ca.`Bike #` = bc.`Bike #`").where("ab.`Bike #` = bc.`Bike #`")\
.where("a.id != b.id").where("b.id != c.id")\
.where("abStart < bcStart").where("bcStart < caStart")\
.orderBy(expr("cast(caStart as long) - cast(abStart as long)"))\
.selectExpr("a.id", "b.id", "c.id", "ab.`Start Date`", "ca.`End Date`")\
.limit(3).show(3, False)

ERROR:root:Exception while sending command.                         (0 + 1) / 4]
Traceback (most recent call last):
  File "/home/f_dev/.local/lib/python3.9/site-packages/py4j/clientserver.py", line 480, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/f_dev/.local/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/f_dev/.local/lib/python3.9/site-packages/py4j/clientserver.py", line 503, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JError: An error occurred while calling o96.showString

In [12]:
# create ring of 4 length
motifs3 = stationGraph.find("(a)-[ab]->(b); (b)-[bc]->(c); (c)-[cb]->(b); (b)-[ba]->(a)")

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
# ring of 4 length query
motifs3.selectExpr("*",
"to_timestamp(ab.`Start Date`, 'MM/dd/yyyy HH:mm') as abStart",
"to_timestamp(bc.`Start Date`, 'MM/dd/yyyy HH:mm') as bcStart",
"to_timestamp(cb.`Start Date`, 'MM/dd/yyyy HH:mm') as cbStart",
"to_timestamp(ba.`Start Date`, 'MM/dd/yyyy HH:mm') as baStart")\
.where("ab.`Bike #` = bc.`Bike #`").where("bc.`Bike #` = cb.`Bike #`").where("cb.`Bike #` = ba.`Bike #`")\
.where("a.id != b.id").where("b.id != c.id")\
.where("abStart < bcStart").where("bcStart < cbStart").where("cbStart < baStart")\
.orderBy(expr("cast(baStart as long) - cast(abStart as long)"))\
.selectExpr("a.id", "b.id", "c.id", "ab.`Start Date`", "ba.`End Date`")\
.limit(4).show(4, False)

In [32]:
#display edges for stations
stationGraph.edges.show(5)

+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
|Trip ID|Duration|     Start Date|                 src|Start Terminal|       End Date|                 dst|End Terminal|Bike #|Subscriber Type|Zip Code|
+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
| 913460|     765|8/31/2015 23:26|Harry Bridges Pla...|            50|8/31/2015 23:39|San Francisco Cal...|          70|   288|     Subscriber|    2139|
| 913459|    1036|8/31/2015 23:11|San Antonio Shopp...|            31|8/31/2015 23:28|Mountain View Cit...|          27|    35|     Subscriber|   95032|
| 913455|     307|8/31/2015 23:13|      Post at Kearny|            47|8/31/2015 23:18|   2nd at South Park|          64|   468|     Subscriber|   94107|
| 913454|     409|8/31/2015 23:10|  San Jose City Hall|            10|8/31/2015 23

In [33]:
# order stations by largest number of other stations
stationGraph.inDegrees.orderBy("inDegree").sort('inDegree', ascending=False).limit(5).show()

+--------------------+--------+
|                  id|inDegree|
+--------------------+--------+
|San Francisco Cal...|   34810|
|San Francisco Cal...|   22523|
|Harry Bridges Pla...|   17810|
|     2nd at Townsend|   15463|
|     Townsend at 7th|   15422|
+--------------------+--------+



In [34]:
# get count for stations groups
from pyspark.sql.functions import col, desc, asc
stationGraph.edges.groupBy("src", 
"dst").count().orderBy(col("count").desc()).show(10)

+--------------------+--------------------+-----+
|                 src|                 dst|count|
+--------------------+--------------------+-----+
|San Francisco Cal...|     Townsend at 7th| 3748|
|Harry Bridges Pla...|Embarcadero at Sa...| 3145|
|     2nd at Townsend|Harry Bridges Pla...| 2973|
|     Townsend at 7th|San Francisco Cal...| 2734|
|Harry Bridges Pla...|     2nd at Townsend| 2640|
|Embarcadero at Fo...|San Francisco Cal...| 2439|
|   Steuart at Market|     2nd at Townsend| 2356|
|Embarcadero at Sa...|   Steuart at Market| 2330|
|     Townsend at 7th|San Francisco Cal...| 2192|
|Temporary Transba...|San Francisco Cal...| 2184|
+--------------------+--------------------+-----+
only showing top 10 rows



In [35]:
stationGraph.edges.groupBy("src").count().orderBy(col("count").desc()).show()

+--------------------+-----+
|                 src|count|
+--------------------+-----+
|San Francisco Cal...|26304|
|San Francisco Cal...|21758|
|Harry Bridges Pla...|17255|
|Temporary Transba...|14436|
|Embarcadero at Sa...|14158|
|     2nd at Townsend|14026|
|     Townsend at 7th|13752|
|   Steuart at Market|13687|
|      Market at 10th|11885|
|   Market at Sansome|11431|
|       Market at 4th| 9894|
|  Powell Street BART| 9695|
|   2nd at South Park| 9469|
|     Beale at Market| 8359|
|Grant Avenue at C...| 8337|
|       2nd at Folsom| 7999|
|Civic Center BART...| 7760|
|       5th at Howard| 7708|
|Broadway St at Ba...| 7676|
|Embarcadero at Fo...| 7596|
+--------------------+-----+
only showing top 20 rows



In [36]:
# count number of triangles passing through
stationGraph.triangleCount().show()

+-----+----------+--------------------+---------+-----------+---------+-------------+------------+
|count|station_id|                  id|      lat|       long|dockcount|     landmark|installation|
+-----+----------+--------------------+---------+-----------+---------+-------------+------------+
|  496|        51|Embarcadero at Fo...|37.791464|-122.391034|       19|San Francisco|   8/20/2013|
|  496|        58|San Francisco Cit...| 37.77865|-122.418235|       19|San Francisco|   8/21/2013|
|  496|        60|Embarcadero at Sa...| 37.80477|-122.403234|       15|San Francisco|   8/21/2013|
|  496|        65|     Townsend at 7th|37.771058|-122.402717|       15|San Francisco|   8/22/2013|
|  496|        63|       Howard at 2nd|37.786978|-122.398108|       19|San Francisco|   8/22/2013|
|   39|        33|Rengstorff Avenue...|37.400241|-122.099076|       15|Mountain View|   8/16/2013|
|   32|        25|Stanford in Redwo...| 37.48537|-122.203288|       15| Redwood City|   8/12/2013|
|  496|   

In [23]:
from graphframes.examples import Graphs
g = Graphs(sqlContext).friends()  # Get example graph

ModuleNotFoundError: No module named 'pyspark.tests'

In [17]:
from neo4j import GraphDatabase

neo_conn = GraphDatabase.driver(uri = 'bolt://localhost:7687', auth = ('neo4j', 'test'))
neo_session = neo_conn.session()

In [22]:
# read from Movie db in neo 4j and import into spark
spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", "bolt://localhost:7687") \
  .option("labels", ":Movie") \
  .load()

Py4JJavaError: An error occurred while calling o101.load.
: org.neo4j.driver.exceptions.AuthenticationException: The client is unauthorized due to authentication failure.
	at org.neo4j.driver.internal.util.Futures.blockingGet(Futures.java:144)
	at org.neo4j.driver.internal.InternalSession.run(InternalSession.java:69)
	at org.neo4j.driver.internal.InternalSession.run(InternalSession.java:51)
	at org.neo4j.driver.internal.AbstractQueryRunner.run(AbstractQueryRunner.java:37)
	at org.neo4j.driver.internal.AbstractQueryRunner.run(AbstractQueryRunner.java:55)
	at org.neo4j.spark.util.ValidateConnection.validate(Validations.scala:69)
	at org.neo4j.spark.util.Validations$.$anonfun$validate$1(Validations.scala:12)
	at org.neo4j.spark.util.Validations$.$anonfun$validate$1$adapted(Validations.scala:12)
	at scala.collection.immutable.Set$Set1.foreach(Set.scala:141)
	at org.neo4j.spark.util.Validations$.validate(Validations.scala:12)
	at org.neo4j.spark.DataSource.inferSchema(DataSource.scala:28)
	